# Welcome to the Notebook for Monha's and Bemi's Bachelor Project

## Content

In this notebook we will:

1. Aggrigate our data into usable travel sequences with only the relevant data 
2. Analyse the appropriate data
3. Create an embedding space using Word2Vec

We will use the following format for the structure of the file:
1. MD file to describe the intention of the following code followed by an explanation of the results from the code if any
2. Code block to write code

# Initial Setup

Please pip install the correct libraries for the following code to work.

In [1]:
%pip install pandas # Pandas for data handling
%pip install numpy  # Maths stuff

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/a5/78/1d859bfb619c067e3353ed079248ae9532c105c4e018fa9a776d04b34572/pandas-2.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for numpy<2,>=1.23.2 from https://files.pythonhosted.org/packages/1a/2e/151484f49fd03944c4a3ad9c418ed193cfd02724e138ac8a9505d056c582/numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 274.2 kB/s eta 0:00:001m248.2 kB/s eta 0:00:01
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import numpy as np

# Data import

TODO:
- Explain the data we use
    - introduction to the DB
    - SJ / RP
    - Journey - Tickets - Orders
- SQL explained and reasoned
- Explain the modification of Lat and Long

---
```SQL
SELECT J.Id as JId, J.CreatedOn, J.SearchStart, J.SearchEnd, J.StartStop, J.EndStop,
    J.StartZone, J.Endzone, J.internalStartZones, J.internalValidZones, 
    SJWaypoints._id as SJId, SJWaypoints.Id,  SJWaypoints.Name, SJWaypoints.Latitude, 
    SJWaypoints.Longitude, SJWaypoints.[Type], SJWaypoints.SJSearchJourney_Id
FROM Journeys J
    JOIN Tickets ON J.Id = Tickets.Journey_Id
    JOIN Orders ON Orders.Id = Tickets.OrderId
    JOIN SJSearchJourneys SJ ON SJ.Id = Orders.JourneyClasses_Id
    JOIN SJWaypoints ON SJWaypoints.SJSearchJourney_Id = SJ.Id
WHERE J.CreatedOn BETWEEN '2022-12-01 00:00:00' and '2023-01-01 00:00:00'
```




In [2]:
data_måned_SJ = pd.read_csv('../Data/JTOSJW_dec_jan_2022.csv')

# data = pd.read_csv('../Data/11mil_large.csv', nrows=8874470)
# data = pd.read_csv('../Data/JTOSJW_dec_jan_2022.csv')
temp1 = pd.read_csv('../Data/Journeys_SearchStartEnd_Dec.csv')
temp2 = pd.read_csv('../Data/Journeys_StartEndStop_Dec.csv')
data_måned_Journeys = pd.concat([temp1,temp2])

data_full_Journeys = pd.read_csv('../Data/ALLJourneysWithValues.csv')

data_måned_SJ = data_måned_SJ.rename(columns={'Endzone':'EndZone'})

# data['Latitude'] = data['Latitude'] / 1000000
# data['Longitude'] = data['Longitude'] /1000000

# data

In [3]:
data_full_Journeys.loc[:, ['internalStartZones', 'StartZone', 'StartStop', 'EndStop', 'SearchStart', 'SearchEnd']]
data_full_Journeys[(data_full_Journeys['StartZone'] == 0)]

,Id,Type,internalStartZones,StartZone,internalValidZones,StartStop,AmountOfZones,EndZone,EndStop,SearchStart,SearchEnd,ModifiedOn,CreatedOn,JourneyClasses_Id,TravelType,ExtraFrom,ExtraTo
0,715ec968-7783-4b6b-be27-0000014f64b3,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,"Hovedbanegården, Tivoli (Bernstorffsgade) (01)","Borrebyvej 29, 2700 Brønshøj, Københavns Kommune",2023-08-18 22.19.25.4586286,2023-08-18 22.19.25.4586286,NaN,NaN,NaN,NaN
1,cbd5ad3b-0bf0-4314-bd74-000001a41c82,NaN,NaN,0,"1001,1002,1003",NaN,3,0,NaN,Femøren St. (Metro),Bispebjerg Hospital (Tagensvej),2023-08-04 08.33.59.0415651,2023-08-04 08.33.59.0415651,NaN,NaN,NaN,NaN
2,afbd1023-61e5-4b47-9f9d-000001c9d32b,Zone,1062,0,"1062,1071,1061,1052,1053,1063,1073,1072,1051,1...",NaN,4,0,NaN,Farum St. (62),Dyssegård St. (31),2023-11-09 17.28.41.0671191,2023-11-09 17.28.41.0671191,NaN,NaN,NaN,NaN
3,2d4257df-2507-47d6-ac81-0000036aa5f2,Zone,1009,0,"1009,1019,1091,1081,1082,1092",NaN,2,0,NaN,Min lokation (09),2 zoner,2022-06-16 10.22.14.3677529,2022-06-16 10.22.14.3677529,NaN,NaN,NaN,NaN
4,1500617d-e006-48c2-b581-000004442f53,Zone,1002,0,"1002,1001,1003,1030,1031,1032,1033",NaN,2,0,NaN,Min lokation (02),2 zoner,2021-04-13 11.39.24.2094151,2021-04-13 11.39.24.2094151,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23527452,d63982ed-13ca-4574-87c2-fffffe18fee2,Zone,1003,0,"1003,1001,1002,1004",NaN,2,0,NaN,Ørestad St. (Ørestads Boulevard) (03),Nørreport St. (01),2024-03-11 15.20.34.2692848,2024-03-11 15.20.34.2692848,NaN,NaN,NaN,NaN
23527453,74795e73-8f89-4b5f-b392-fffffe36c3d3,Zone,1002,0,"1002,1001,1003,1030,1031,1032,1033,1004,1040,1...",NaN,4,0,NaN,Min Lokation (02),Måløv St. (53),2021-01-05 13.41.15.8483902,2021-01-05 13.41.15.8483902,NaN,NaN,NaN,NaN
23527454,4afd6d02-6599-4345-8fc5-fffffe936d49,Zone,1002,0,"1002,1001,1003,1030,1031,1032,1033,1004,1040,1...",NaN,3,0,NaN,Hans Knudsens Plads (Lyngbyvej) (02),Lyngby St. (41/51),2023-06-08 08.31.27.5187233,2023-06-08 08.31.27.5187233,NaN,NaN,NaN,NaN
23527455,2c273886-33bf-4234-b58c-fffffeb987cf,Zone,1002,0,"1002,1001,1003,1030,1031,1032,1033",NaN,2,0,NaN,Ryparken St.,Nordhavn St.,2022-03-14 20.46.13.2072936,2022-03-14 20.46.13.2072936,NaN,NaN,NaN,NaN


# Analysis of the data

Since or data consists only of a certain 'ticket'-type (DOT-ticket) we wish to analyse the enterierty of our data. For the analysis certain questions serves as our startingpoint:
1. Who actually uses this ticketsystem?
2. How representative in the context of CPH is our data?
3. ...

# Prepare data for Word2Vec

We now wish to transform our data into journey-sequences which we in turn can use to train a model using Word2Vec. 

For this to work as intended we wish to transform our data such that:
1. Only journeys WITHIN CPH is present
2. A data entry consists of a sequence of stops for a given journey
    - The stops should consist of Start, End and Transitional stops
    - We wish to make use of a dictionary 
        - REASONING FOR DICT


## Copenhagen filter

In order to filter our data, 3 checks need to be made to be certain a journey is within cph. For a journey to be within cph they need to only make use of zone 1 through 4
1. Check if *StartZone* and *EndZone* is within 1 and 4
2. Check if *internalStartZones* only contain zones within cph
3. Check if *internalValidZones* only contain zones within cph

In [6]:
def get_conditions(data):

    condition_1 = (
        (data['StartZone'].between(1001, 1004) | pd.isna(data['StartZone']))
        & 
        (data['EndZone'].between(1001, 1004) | pd.isna(data['EndZone']))
        )

    # THIS CONDITION MIGHT NOT BE RELEVANT DUE TO THE TICKETING SYSTEM OF DOT-BILLET
    # SINCE WHEN YOU BUY A TICKET IT IS USUALLY VALID IN ~2+ ZONES FROM WHERE YOU START. THIS MEANS
    # IF YOU START IN ZONE 2, THEN ZONE 3x, 1 AND 3 IS ALL A 'VALID ZONE' - THAT IS, THE START ZONE'S ADJACENT ZONES
    # WE ARE NOT REALLY INTERESTED IN WHAT ZONES A TICKET IS VALID IN, AS LONG AS WE CAN BE SURE, THAT THE JOURNEY
    # ONLY TOOK PLACE INSIDE OF COPENHAGEN -> ZONE 1 TO 4.
    condition_2 = (
        (data['internalValidZones'].str.match(r'^(1001|1002|1003|1004)(,(1001|1002|1003|1004))*$')
        | # or
        pd.isna(data['internalValidZones']))
        )

    condition_3 = (
        (data['internalStartZones'].str.match(r'^(1001|1002|1003|1004)$'))
        | # or
        pd.isna(data['internalStartZones'])
        )
    return (condition_2 | condition_3)

cph_data_SJ = data_måned_SJ[get_conditions(data_måned_SJ)]
cph_data_Journeys = data_måned_Journeys[get_conditions(data_måned_Journeys)]

condition_2 = (
    (data_full_Journeys['internalValidZones'].str.match(r'^(1001|1002|1003|1004)(,(1001|1002|1003|1004))*$')
    | # or
    pd.isna(data_full_Journeys['internalValidZones']))
    )

condition_3 = (
    (data_full_Journeys['internalStartZones'].str.match(r'^(1001|1002|1003|1004)$'))
    | # or
    pd.isna(data_full_Journeys['internalStartZones'])
    )


cph_data_Journeys_full_1 = data_full_Journeys[(condition_2)]
cph_data_Journeys_full_2 = cph_data_Journeys_full_1[condition_3]

# grp = cph_data.groupby('SJSearchJourney_Id').agg(list)

# grp.count()

# grp.count()

# cph_data_SJ

/var/folders/6c/gl4j6k8x3tn79jcjxtk_ly1h0000gn/T/ipykernel_90136/743106485.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cph_data_Journeys_full_2 = cph_data_Journeys_full_1[condition_3]


# How many waypoint-journeys in copenhagen? 

7790

In [30]:
condition_2_SJ = (
    (data_måned_SJ['internalValidZones'].str.match(r'^(1001|1002|1003|1004)(,(1001|1002|1003|1004))*$')
    | # or
    pd.isna(data_måned_SJ['internalValidZones']))
    )

condition_3_SJ = (
    (data_måned_SJ['internalStartZones'].str.match(r'^(1001|1002|1003|1004)$'))
    | # or
    pd.isna(data_måned_SJ['internalStartZones'])
    )

cph_data_SJ_full_1 = data_måned_SJ[(condition_2_SJ)]
cph_data_SJ_full_2 = cph_data_SJ_full_1[condition_3_SJ]
cph_data_SJ_full_2.groupby(cph_data_SJ_full_2['JId']).agg(list).count()

/var/folders/6c/gl4j6k8x3tn79jcjxtk_ly1h0000gn/T/ipykernel_90136/3413264281.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cph_data_SJ_full_2 = cph_data_SJ_full_1[condition_3_SJ]


CreatedOn             7790
SearchStart           7790
SearchEnd             7790
StartStop             7790
EndStop               7790
StartZone             7790
EndZone               7790
internalStartZones    7790
internalValidZones    7790
SJId                  7790
Id                    7790
Name                  7790
Latitude              7790
Longitude             7790
Type                  7790
SJSearchJourney_Id    7790
dtype: int64

In [7]:
# grp_2 = cph_data_SJ.groupby('JId').agg(list)
# grp_2.count()

# data_måned_SJ.groupby('JId').agg(list).count()

# cph_data_Journeys.groupby('Id').agg(list).count()
# cph_data.count()

# data.count()

cph_data_Journeys_full_2.count()



Id                    8092740
Type                  8092699
internalStartZones    8092740
StartZone             8092740
internalValidZones    8092740
StartStop                 360
AmountOfZones         8092740
EndZone               8092740
EndStop                   360
SearchStart           8092406
SearchEnd             7617012
ModifiedOn            8092740
CreatedOn             8092740
JourneyClasses_Id           0
TravelType                  0
ExtraFrom                   0
ExtraTo                     0
dtype: int64

In [18]:
# data_måned_Journeys[(data_måned_Journeys['SearchEnd'] == '2 zoner')] # 310
# data_måned_Journeys[data_måned_Journeys['SearchStart'].str.contains("okation", na=False)] # 419
# data_filter_1 = data_måned_Journeys[~data_måned_Journeys['SearchStart'].str.contains("okation", na=False) | ~data_måned_Journeys['SearchEnd'].str.contains("zone", na=False)] # 419
# data_filter_2 = data_filter_1[(data_filter_1['SearchStart'] != data_filter_1['SearchEnd'])  | (data_filter_1['StartStop'] != data_filter_1['EndStop'])] # 419
# data_filter_2


# First two filters are in order to remove entries from the table where 'lokation' and 'zoner' are the values
data_filter_1_full = cph_data_Journeys_full_2[ ~ (cph_data_Journeys_full_2['SearchStart'].str.contains("okation", na=False)
                                             | 
                                             cph_data_Journeys_full_2['SearchStart'].str.contains("zoner", na=False))]
data_filter_2_full = data_filter_1_full[( ~ (data_filter_1_full['SearchEnd'].str.contains("zoner", na=False) 
                                            |
                                            data_filter_1_full['SearchEnd'].str.contains("okation", na=False)))]
# next two filters are English filters of the first
data_filter_3_full = data_filter_2_full[( ~ (data_filter_2_full['SearchEnd'].str.contains("zones", na=False) 
                                            |
                                            data_filter_2_full['SearchEnd'].str.contains("ocation", na=False)))]

data_filter_4_full = data_filter_3_full[( ~ (data_filter_3_full['SearchStart'].str.contains("zones", na=False) 
                                            |
                                            data_filter_3_full['SearchStart'].str.contains("ocation", na=False)))]
# Next filter is to remove entries where one of the matching search-x or x-stop are Null
data_filter_5_full = data_filter_4_full[(
                                        ( ~ (pd.isna(data_filter_4_full['SearchStart'])) & ~ (pd.isna(data_filter_4_full['SearchEnd'])))
                                        | # Or
                                        ( ~ (pd.isna(data_filter_4_full['StartStop'])) & ~ (pd.isna(data_filter_4_full['EndStop'])))
                                        )]
data_filter_5_full


# data_filter_2_full[(pd.isna(data_filter_2_full['StartStop']))] #I vores data for alle journeys har vi 1138 rejser i kbh. Her er 681 af dem med start og slut stop, men 457 af dem uden hvor vi må stole search


# data_måned_SJ[data_måned_SJ['SearchStart'].str.contains("okation", na=False) | data_måned_SJ['SearchEnd'].str.contains("zone", na=False)] # 419

,Id,Type,internalStartZones,StartZone,internalValidZones,StartStop,AmountOfZones,EndZone,EndStop,SearchStart,SearchEnd,ModifiedOn,CreatedOn,JourneyClasses_Id,TravelType,ExtraFrom,ExtraTo
0,715ec968-7783-4b6b-be27-0000014f64b3,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,"Hovedbanegården, Tivoli (Bernstorffsgade) (01)","Borrebyvej 29, 2700 Brønshøj, Københavns Kommune",2023-08-18 22.19.25.4586286,2023-08-18 22.19.25.4586286,NaN,NaN,NaN,NaN
12,27ddd4c7-35d5-4e95-84a5-0000092e14a0,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,København H (togbus) (01),Hulgårds Plads (Frederikssundsvej) (02),2023-01-01 13.16.11.4343765,2023-01-01 13.16.11.4343765,NaN,NaN,NaN,NaN
30,986798c5-e47a-4a6f-ba89-0000177df4cf,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,København H (togbus) (01),Islands Brygge St. (Metro) (01),2022-08-25 09.08.46.4521964,2022-08-25 09.08.46.4521964,NaN,NaN,NaN,NaN
35,ad7d6db6-ab4e-4782-8976-000019f0ecd6,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,København H (Metro) (01),Frederiksberg Allé St. (Metro) (01),2023-07-24 05.41.23.6936628,2023-07-24 05.41.23.6936628,NaN,NaN,NaN,NaN
48,fec8331d-e54e-48b2-88fe-00002217de59,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,Nørreport St. (01),Sluseholmen (Sjællandsbroen) (02),2022-10-05 12.44.54.5227363,2022-10-05 12.44.54.5227363,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23527424,7b4d55b7-a4f5-453c-887f-ffffe57690b7,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,Amagerbro St. (Metro) (01),Kongens Nytorv St. (Metro) (01),2024-01-31 06.42.46.8616806,2024-01-31 06.42.46.8616806,NaN,NaN,NaN,NaN
23527431,490baace-c75d-4fda-b608-ffffee254091,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,Islands Brygge St. (Ørestads Boulevard),Forum St. (Metro),2023-08-23 12.08.49.4881799,2023-08-23 12.08.49.4881799,NaN,NaN,NaN,NaN
23527432,1bb14585-68a2-404d-bc36-ffffeef1aa01,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,Rådhuspladsen St. (Vesterbrogade) (01),Skt. Annæ Gade (Prinsessegade) (01),2024-01-13 18.51.35.6627821,2024-01-13 18.51.35.6627821,NaN,NaN,NaN,NaN
23527440,fb57446e-412e-4365-8656-fffff4b08b3d,Zone,1001,0,"1001,1002,1003",NaN,2,0,NaN,"Hovedbanegården, Frihedsstøtten (Vesterbrogade...","Roskildevej 96, 2000 Frederiksberg, Frederiksb...",2024-02-14 15.59.19.9272062,2024-02-14 15.59.19.9272062,NaN,NaN,NaN,NaN


In [23]:
data_filter_4_full[data_filter_2_full['SearchEnd'].str.contains("okation", na=False)]
data_filter_4_full[data_filter_2_full['SearchEnd'].str.contains("zones", na=False)]
data_filter_4_full[data_filter_2_full['SearchStart'].str.contains("zones", na=False)] # There was one hmmm
data_filter_5_full[~pd.isna(data_filter_2_full['StartStop'])]


/var/folders/6c/gl4j6k8x3tn79jcjxtk_ly1h0000gn/T/ipykernel_90136/115260756.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_filter_4_full[data_filter_2_full['SearchEnd'].str.contains("okation", na=False)]
/var/folders/6c/gl4j6k8x3tn79jcjxtk_ly1h0000gn/T/ipykernel_90136/115260756.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_filter_4_full[data_filter_2_full['SearchEnd'].str.contains("zones", na=False)]
/var/folders/6c/gl4j6k8x3tn79jcjxtk_ly1h0000gn/T/ipykernel_90136/115260756.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_filter_4_full[data_filter_2_full['SearchStart'].str.contains("zones", na=False)] # There was one hmmm
/var/folders/6c/gl4j6k8x3tn79jcjxtk_ly1h0000gn/T/ipykernel_90136/115260756.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_filter_5_full[~pd.isna(data_filter_2_full['StartStop'])]


,Id,Type,internalStartZones,StartZone,internalValidZones,StartStop,AmountOfZones,EndZone,EndStop,SearchStart,SearchEnd,ModifiedOn,CreatedOn,JourneyClasses_Id,TravelType,ExtraFrom,ExtraTo
545223,be6d19a3-6e7c-4092-918e-05eec18edaa5,Relation,1004,1004,"1004,1003,1001,1002",8600858.0,3,1001,8600626.0,Lufthavnen St. (Metro) (04),København H (01),2022-09-11 09.17.42.7782230,2022-09-11 09.17.42.7782230,NaN,NaN,NaN,NaN
708157,24cedb1c-8bc1-4a6b-b0ff-07b390a47445,Relation,1004,1004,"1004,1003,1001,1002",8600858.0,3,1001,8600626.0,"Kastrup Tværvej E 5, 2770 Kastrup, Tårnby Kommune","Hovedbanegården, Tivoli (Bernstorffsgade) (01)",2022-10-28 12.41.03.1232798,2022-10-28 12.41.03.1232798,NaN,NaN,NaN,NaN
866164,628ef453-82b8-444b-9409-096ad9f5ab4f,Relation,1001,1001,"1001,1002,1003",8600626.0,2,1003,8600856.0,København H (Metro) (01),Ørestad St. (03),2022-07-29 14.16.19.2847891,2022-07-29 14.16.19.2847891,NaN,NaN,NaN,NaN
955362,c091ff32-d730-442d-87d2-0a62840af2e0,Relation,1004,1004,"1004,1003,1001,1002",8600858.0,3,1001,8600626.0,Lufthavnen St. (Metro) (04),København H (01),2022-11-12 11.33.57.0075902,2022-11-12 11.33.57.0075902,NaN,NaN,NaN,NaN
989748,f4352b66-9e75-4157-a215-0ac274b6bcdd,Relation,1004,1004,"1004,1003,1001,1002",8600858.0,3,1001,8600626.0,CPH Lufthavn (04),København H (01),2022-12-16 14.43.33.9685201,2022-12-16 14.43.33.9685201,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22902823,4f0e05e6-c971-4a3a-aadf-f93577dc7179,Relation,1004,1004,"1004,1003,1001,1002",8600858.0,3,1001,8600626.0,CPH Lufthavn (04),København H (01),2022-09-09 05.53.36.9909461,2022-09-09 05.53.36.9909461,NaN,NaN,NaN,NaN
23116253,abf929ae-9381-42a5-9a01-fb87bfabb0d9,Relation,1003,1003,"1003,1001,1002,1004",8600856.0,2,1004,8600858.0,Ørestad St.,CPH Lufthavn,2021-09-15 22.19.03.6880966,2021-09-15 22.19.03.6880966,NaN,NaN,NaN,NaN
23150445,07f1ff23-edd5-4a46-b2e3-fbe6f6b1ed19,Relation,1001,1001,"1001,1002,1003",8603321.0,2,1002,8603301.0,Amagerbro St. (Metro),Vanløse St.,2020-08-25 17.26.26.5456076,2020-08-25 17.26.26.5456076,NaN,NaN,NaN,NaN
23317340,595937d1-2f25-4e7b-a3a9-fdb826e5a033,Relation,1003,1003,"1003,1001,1002,1004",8600856.0,2,1004,8600857.0,NaN,NaN,2023-02-28 13.28.07.6798877,2020-08-21 20.23.55.7341230,NaN,NaN,NaN,NaN


# Code used for testing purposes

DISCLAIMER:

NOTEBOOK IS MEMORY BASED SO CERTAIN ELEMENTS OF THE CODE BELOW MIGHT NOT WORK IF RUN AGAIN. LOOK AT THE COMMENETED PRINT STATEMENTS NEXT TO THE CODE FOR THE RELEVANT OUTPUTS AT THE TIME

In [20]:
# grp = data.groupby('SJSearchJourney_Id').agg(list)

def not_null(lst):
    for value in lst:
        if value is None or pd.isna(value):
            return False
    return True

# We wish to check if the amount of StartZones match the amount of Endzones:
print(grp['StartZone'].apply(not_null).sum()) # 31965
print(grp['EndZone'].apply(not_null).sum()) # 31965

# print(grp[(grp['StartZone'].apply(not_null) & grp['EndZone'].apply(not_null))]) # 31965 rows


def digigt_above_1000(lst):
    for elem in lst:
        if elem < 1000:
            False
    return True

print(grp['StartZone'].apply(digigt_above_1000).sum()) # 31965


# def not_contain_comma(lst: list[str]):
#     for elem in lst:
#         if ',' in elem:
#             False
#     return True
# print(grp['internalStartZones'].apply(not_contain_comma).sum()) # 31965 - indicating that all internalStartZones only consists of a single zone


def not_contain_space(lst: list[str]):
    for elem in lst:
        if ' ' in elem:
            False
    return True
print(grp['internalValidZones'].apply(not_contain_space).sum()) # 31965 - indicating that all internalStartZones only consists of a single zone

508320
508320
508320
508320
